1. Створити кілька своїх прикладів у форматі json за тематикою варіанту (англійською або українською мовою) для розпізнавання нового типу сутностей (обрати самостійно). Створити програму, що додає ці приклади до існуючої моделі spaCy, навчає модель. Продемонструвати роботу.
2. Застосувати компонент TextCategorizer для визначення намірів. Дані для навчання за тематикою варіанту обрати самостійно або скористатись вказаним файлом (utterance містить висловлювання, intent - намір). Дані файли містять приклади діалогів користувачів з системою-помічником за певною тематикою, наприклад, замовлення квитків і т.д. Навчити компонент та продемонструвати роботу.

Тематика: таксі та оренда машин. Файл taxiandcars.json.

## Зчитування json файлу з навчальними прикладами за тематикою таксі та оренди машин (англійською мовою)

In [1]:
import spacy
from spacy.training import Example
import random
import json

nlp = spacy.load('en_core_web_md')

with open('train_data.json') as file:
    train_data_json = json.load(file)

train_data = []
for item in train_data_json:
    sentence = item['sentence']
    value = item['entities']['value']
    start = sentence.index(value)
    end = start + len(value)
    train_data.append((sentence, {'entities': [(start, end, item['entities']['label'])]}))
    
train_data[:3]

[('I rented a car from Hertz last weekend.',
  {'entities': [(20, 25, 'CAR_RENTAL_COMPANY')]}),
 ('We booked our vehicle through Avis for the holiday.',
  {'entities': [(30, 34, 'CAR_RENTAL_COMPANY')]}),
 ('Enterprise provided excellent service during our trip.',
  {'entities': [(0, 10, 'CAR_RENTAL_COMPANY')]})]

## Навчання моделі spaCy для розпізнавання нового типу сутностей

In [2]:
epochs = 20
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.create_optimizer()
    for i in range(epochs):
        random.shuffle(train_data)
        for text, annotations in train_data:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], sgd=optimizer)

ner = nlp.get_pipe('ner')
ner.to_disk('new_ner_model')

## Демонстрація навченої моделі

In [3]:
test_texts = [
    'I need to rent a car from Hertz for my trip.',
    'Can you help me book a vehicle through Avis?',
    'We had a pleasant experience renting from Enterprise.',
    'Hertz provided the best car rental service.',
    "I'm looking to book a car rental with Avis for next week."
]

for text in test_texts:
    doc = nlp(text)
    print(f'Text: {text}')
    for ent in doc.ents:
        print(f'Entity: {ent.text}, Label: {ent.label_}')
    print()

Text: I need to rent a car from Hertz for my trip.
Entity: Hertz, Label: CAR_RENTAL_COMPANY

Text: Can you help me book a vehicle through Avis?
Entity: Avis, Label: CAR_RENTAL_COMPANY

Text: We had a pleasant experience renting from Enterprise.
Entity: Enterprise, Label: CAR_RENTAL_COMPANY

Text: Hertz provided the best car rental service.
Entity: Hertz, Label: CAR_RENTAL_COMPANY

Text: I'm looking to book a car rental with Avis for next week.
Entity: Avis, Label: CAR_RENTAL_COMPANY


## Зчитування json файлу діалогів користувачів з системою-помічником

In [4]:
with open('taxiandcars.json') as file:
    data = json.load(file)
    
train_data = []
for dialogue in data:
    for turn in dialogue['turns']:
        if 'frames' in turn and turn['frames']:
            frame = turn['frames'][0]
            if 'state' in frame and 'active_intent' in frame['state'] and 'utterance' in turn:
                intent = frame['state']['active_intent']
                utterance = turn['utterance']
                train_data.append((utterance, {'cats': {intent: 1}}))

## Застосування компоненту TextCategorizer для визначення намірів

In [5]:
from spacy.pipeline.textcat import Config, DEFAULT_SINGLE_TEXTCAT_MODEL
from spacy.training import Example

nlp = spacy.blank('en')
config = Config({'model': DEFAULT_SINGLE_TEXTCAT_MODEL, 'threshold': 0.5})
textcat = nlp.add_pipe('textcat', config=config)
for _, annotations in train_data:
    for cat in annotations['cats'].keys():
        textcat.add_label(cat)

train_examples = [Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in train_data]
textcat.initialize(lambda: train_examples, nlp=nlp)

epochs = 20
optimizer = nlp.create_optimizer()
for i in range(epochs):
    random.shuffle(train_data)
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], sgd=optimizer)

## Демонстрація роботи TextCategorizer

In [6]:
test_texts = [
    "I need a cab for two people.",
    "Can you book a car rental for me?",
    "I would like to rent a car for my vacation.",
    "Please call me a taxi.",
    "I need to go to the airport."
]

for test_text in test_texts:
    doc = nlp(test_text)
    print('-' * 50)
    print(f"Text: {test_text}\nMost relevant category: {max(doc.cats, key=doc.cats.get)}")

--------------------------------------------------
Text: I need a cab for two people.
Most relevant category: GetRide
--------------------------------------------------
Text: Can you book a car rental for me?
Most relevant category: ReserveCar
--------------------------------------------------
Text: I would like to rent a car for my vacation.
Most relevant category: GetCarsAvailable
--------------------------------------------------
Text: Please call me a taxi.
Most relevant category: GetRide
--------------------------------------------------
Text: I need to go to the airport.
Most relevant category: GetRide
